In [ ]:
# %pip install -U langchain langchain-community

In [ ]:
# %pip install faiss-cpu

In [ ]:
# %pip install transformers sentence-transformers

In [ ]:
# %pip install rank_bm25

## Retriever

In [ ]:
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
import numpy as np
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from sentence_transformers import CrossEncoder
from typing import List, Dict
import torch

In [ ]:
embedder_labse = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru"
)

<ipython-input-14-5f9169f6a361>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder_labse = HuggingFaceEmbeddings(


In [ ]:
embedder_mini = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
embedder_multi = HuggingFaceEmbeddings(
     model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

In [ ]:
vector_store_labse = FAISS.load_local("faiss_labse_all", embedder_labse, allow_dangerous_deserialization=True)

In [ ]:
vector_store_mini = FAISS.load_local("faiss_mini", embedder_mini, allow_dangerous_deserialization=True)

In [ ]:
vector_store_multi = FAISS.load_local("faiss_multi", embedder_multi, allow_dangerous_deserialization=True)

In [ ]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder("amberoad/bert-multilingual-passage-reranking-msmarco", max_length=2048)

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
bm25_retriever_labse = BM25Retriever.from_documents(list(vector_store_labse.docstore._dict.values()), metadata_keys=["document", "part"])

In [ ]:
bm25_retriever_mini = BM25Retriever.from_documents(list(vector_store_mini.docstore._dict.values()), metadata_keys=["document", "part"])

In [ ]:
bm25_retriever_multi = BM25Retriever.from_documents(list(vector_store_multi.docstore._dict.values()), metadata_keys=["document", "part"])

In [ ]:
class HierarchicalRetriever:
    def __init__(self, vector_db, bm25_retriever, reranker):
        self.faiss_k = 5
        self.bm25_k = 5
        self.weights = [0.4, 0.6]
        self.vector_db = vector_db
        self.bm25_retriever = bm25_retriever
        self.bm25_retriever.k = self.bm25_k
        self.faiss_retriever = vector_db.as_retriever(search_kwargs={"k":self.faiss_k})
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.faiss_retriever],
            weights=self.weights,
            k=self.bm25_k
        )
        self.reranker = reranker

    def ensemble_retrieve(
        self,
        query: str,
    ) -> List[Document]:

        return self.ensemble_retriever.invoke(query)

    def faiss_retrieve(
        self,
        query: str,
    ) -> List[Document]:

        return self.faiss_retriever.invoke(query)

    def bm25_retrieve(
            self,
            query: str,
    ) -> List[Document]:

        return self.bm25_retriever.invoke(query)

    def retrieve(
        self,
        query: str,
        document_filter: str = None,
        part_keywords: List[str] = None
    ) -> List[Document]:


        docs = self.ensemble_retrieve(query)
        rerank_docs = self.rerank_documents(query, docs, top_k=5)

        part_keyword = rerank_docs[0].metadata['part']

        # Уровень 1: Фильтрация по документу
        if document_filter:
            docs = self.vector_db.similarity_search(
                query,
                filter=lambda doc: doc.metadata["document"] == document_filter
            )
        else:
            docs = self.vector_db.similarity_search(query)

        # Уровень 2: Фильтрация по разделу
        if part_keywords:
            filtered = []
            for doc in docs:
                if any(kw in doc.metadata["part"] for kw in part_keywords):
                    filtered.append(doc)
            docs = filtered

            rerank_docs = docs

        return rerank_docs

    def rerank_documents(
            self,
            query: str,
            docs: list[Document],
            top_k: int = 5
        ) -> list[Document]:

        pairs = [(query, doc.page_content) for doc in docs]

        scores = self.reranker.predict(pairs, batch_size=32)

        scored_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

        return [doc for doc, score in scored_docs[:top_k]]

In [ ]:
custom_retriever_labse = HierarchicalRetriever(vector_store_labse, bm25_retriever_labse, None)
custom_retriever_mini = HierarchicalRetriever(vector_store_mini, bm25_retriever_mini, reranker)
custom_retriever_multi = HierarchicalRetriever(vector_store_multi, bm25_retriever_multi, reranker)

In [ ]:
custom_retriever_labse.faiss_retrieve("Гражданское право даёт человеку")[:1]

[Document(id='dc73eff5-66aa-4644-81f0-705485dcd74f', metadata={'document': 'ГК_merged', 'part': 'Глава 3. Граждане (физические лица) Статья 17. Правоспособность гражданина'}, page_content='Глава 3. Граждане (физические лица) Статья 17. Правоспособность гражданина 1. Способность иметь гражданские права и нести обязанности (гражданская правоспособность) признается в равной мере за всеми гражданами.2. Правоспособность гражданина возникает в момент его рождения и прекращается смертью.')]

In [ ]:
custom_retriever_labse.bm25_retrieve("Гражданское право даёт человеку")[:1]

[Document(id='e7691249-1c07-48d6-81f0-e9a3226b12ea', metadata={'document': 'ГПК', 'part': 'Глава 1. Основные положения Статья 9. Язык гражданского судопроизводства'}, page_content='Глава 1. Основные положения Статья 9. Язык гражданского судопроизводства 1. Гражданское судопроизводство ведется на русском языке - государственном языке Российской Федерации или на государственном языке республики, которая входит в состав Российской Федерации и на территории которой находится соответствующий суд. В военных судах гражданское судопроизводство ведется на русском языке.2. Лицам, участвующим в деле и не владеющим языком, на котором ведется гражданское судопроизводство, разъясняется и обеспечивается право давать объяснения, заключения, выступать, заявлять ходатайства, подавать жалобы на родном языке или на любом свободно избранном языке общения, а также пользоваться услугами переводчика.')]

In [ ]:
custom_retriever_labse.ensemble_retrieve("Гражданское право даёт человеку")[:1]

[Document(id='dc73eff5-66aa-4644-81f0-705485dcd74f', metadata={'document': 'ГК_merged', 'part': 'Глава 3. Граждане (физические лица) Статья 17. Правоспособность гражданина'}, page_content='Глава 3. Граждане (физические лица) Статья 17. Правоспособность гражданина 1. Способность иметь гражданские права и нести обязанности (гражданская правоспособность) признается в равной мере за всеми гражданами.2. Правоспособность гражданина возникает в момент его рождения и прекращается смертью.')]

In [ ]:
custom_retriever_labse.retrieve("Гражданское право даёт человеку")[:1]

[Document(id='59fb3dea-b148-411b-a6fb-9ecf038eb45a', metadata={'document': 'ГК_merged', 'part': 'Глава 1. Гражданское законодательство Статья 5. Обычаи'}, page_content='Глава 1. Гражданское законодательство Статья 5. Обычаи 1. Обычаем признается сложившееся и широко применяемое в какой-либо области предпринимательской или иной деятельности, не предусмотренное законодательством правило поведения, независимо от того, зафиксировано ли оно в каком-либо документе.2. Обычаи, противоречащие обязательным для участников соответствующего отношения положениям законодательства или договору, не применяются.')]

## Загрузка валидационного датасета

In [ ]:
import pandas as pd
test_df = pd.read_csv("test_df", index_col=0)
test_df

,instruction,input,output
2588,Ты - юридический консультант. Тебе даётся след...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ..."
6365,Ты - юридический консультант. Тебе даётся след...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано..."
10807,Ты - юридический консультант. Тебе даётся след...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ..."
1120,Ты - юридический консультант. Тебе даётся след...,"На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ..."
204,Ты - юридический консультант. Тебе даётся след...,Может ли страховая компания взыскать выплаченн...,"Да, может. Согласно п. 1 ст. 14 Федерального з..."
...,...,...,...
6997,Ты - юридический консультант. Тебе даётся след...,"Какие расходы, помимо стоимости ремонта, я мог...",Вы можете взыскать: \n- Расходы на экспертизу...
1227,Ты - юридический консультант. Тебе даётся след...,Какие доказательства нужны для признания имуще...,"Необходимы документы, подтверждающие, что собс..."
3381,Ты - юридический консультант. Тебе даётся след...,"Можно ли оспорить протокол, если я не согласен...","Да, можно попытаться оспорить, если есть доказ..."
11549,Ты - юридический консультант. Тебе даётся след...,Могут ли мои претензии по поводу сделки с квар...,"Да, если ваши высказывания содержали унизитель..."


In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
for custom_retriever in [custom_retriever_labse, custom_retriever_mini, custom_retriever_multi]:
  faiss_score, bm25_score, ensemble_score, custom_score = [],[],[],[]
  retrievers = [
      (custom_retriever.faiss_retrieve, faiss_score),
      (custom_retriever.bm25_retrieve, bm25_score),
      (custom_retriever.ensemble_retrieve,ensemble_score),
      (custom_retriever.retrieve, custom_score)
      ]

  # Получение соответствия ID документа -> эмбеддинг
  doc_embeddings = {}
  for idx, (id, doc) in enumerate(custom_retriever.vector_db.docstore._dict.items()):
    embedding = custom_retriever.vector_db.index.reconstruct(idx)
    doc_embeddings[id] = embedding

  for query in tqdm(test_df['input']):
    for retriever, score in retrievers:
      docs = retriever(query)
      embeddings = np.array([doc_embeddings[doc.id] for doc in docs])
      kmeans = KMeans(n_clusters=2)
      labels = kmeans.fit_predict(embeddings)

      score.append(silhouette_score(embeddings, labels))
  print(f"""
  Faiss: {np.mean(faiss_score)},
  BM25: {np.mean(bm25_score)},
  Ensemble: {np.mean(ensemble_score)},
  Custom: {np.mean(custom_score)}
        """)


100%|██████████| 120/120 [21:28<00:00, 10.74s/it]



  Faiss: 0.11677444726228714,
  BM25: 0.12897588312625885,
  Ensemble: 0.10815028846263885,
  Custom: 0.10638199001550674
        


100%|██████████| 120/120 [20:07<00:00, 10.06s/it]



  Faiss: 0.07460319995880127,
  BM25: 0.1208997443318367,
  Ensemble: 0.2602393925189972,
  Custom: 0.18015196919441223
        


100%|██████████| 120/120 [20:09<00:00, 10.08s/it]


  Faiss: 0.14526067674160004,
  BM25: 0.16816221177577972,
  Ensemble: 0.17548224329948425,
  Custom: 0.18855580687522888
        


In [ ]:
np.mean(faiss_score), np.mean(bm25_score), np.mean(ensemble_score),np.mean(custom_score)

(np.float32(0.10877491),
 np.float32(0.1284074),
 np.float32(0.10790478),
 np.float32(0.11146192))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
for custom_retriever in [custom_retriever_labse, custom_retriever_mini, custom_retriever_multi]:
  faiss_score, bm25_score, ensemble_score, custom_score = [],[],[],[]
  retrievers = [
      (custom_retriever.faiss_retrieve, faiss_score),
      (custom_retriever.bm25_retrieve, bm25_score),
      (custom_retriever.ensemble_retrieve,ensemble_score),
      (custom_retriever.retrieve, custom_score)
      ]

  # Получение соответствия ID документа -> эмбеддинг
  doc_embeddings = {}
  for idx, (id, doc) in enumerate(custom_retriever.vector_db.docstore._dict.items()):
    embedding = custom_retriever.vector_db.index.reconstruct(idx)
    doc_embeddings[id] = embedding

  for query in tqdm(test_df['input']):
    for retriever, score in retrievers:
      docs = retriever(query)
      embeddings = np.array([doc_embeddings[doc.id] for doc in docs])
      sim_matrix = cosine_similarity(embeddings)
      np.fill_diagonal(sim_matrix, 0)

      score.append(1 - sim_matrix.max())
  print(f"""
  Faiss: {np.mean(faiss_score)},
  BM25: {np.mean(bm25_score)},
  Ensemble: {np.mean(ensemble_score)},
  Custom: {np.mean(custom_score)}
        """)


100%|██████████| 120/120 [20:54<00:00, 10.46s/it]



  Faiss: 0.182405024766922,
  BM25: 0.18292300403118134,
  Ensemble: 0.13657788932323456,
  Custom: 0.19875545799732208
        


100%|██████████| 120/120 [21:15<00:00, 10.63s/it]



  Faiss: 0.2133166342973709,
  BM25: 0.11009983718395233,
  Ensemble: 0.10739615559577942,
  Custom: 0.11756580322980881
        


100%|██████████| 120/120 [21:40<00:00, 10.84s/it]


  Faiss: 0.1244497001171112,
  BM25: 0.1853252798318863,
  Ensemble: 0.09896865487098694,
  Custom: 0.16229070723056793
        


In [ ]:
import pandas as pd
model_answers = pd.read_csv("model_answers", index_col=0)
model_answers.head()

,Unnamed: 0,instruction,input,output,model_answer
0,2588,Ты - юридический консультант. Тебе даётся след...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ...","Нет, фотофиксация не обязательна, если иные до..."
1,6365,Ты - юридический консультант. Тебе даётся след...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано...","Да, согласно ч. 1 ст. 30.7 КоАП РФ, суд вправе..."
2,10807,Ты - юридический консультант. Тебе даётся след...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ...","Нет, ст. 5.61 КоАП РФ не предусматривает отягч..."
3,1120,Ты - юридический консультант. Тебе даётся след...,"На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ...","По п. 21 ст. 12 Закона об ОСАГО, за каждый ден..."
4,204,Ты - юридический консультант. Тебе даётся след...,Может ли страховая компания взыскать выплаченн...,"Да, может. Согласно п. 1 ст. 14 Федерального з...","Да, согласно п. «г» ч. 1 ст. 14 Федерального з..."


In [ ]:
from tqdm import tqdm

In [ ]:
for custom_retriever, retirever_name in [(custom_retriever_labse, 'labse'), (custom_retriever_mini, "mini"), (custom_retriever_multi, "multi")]:

  retrievers = [
      (custom_retriever.faiss_retrieve, "faiss"),
      (custom_retriever.bm25_retrieve, "bm25"),
      (custom_retriever.ensemble_retrieve,"ensemble"),
      (custom_retriever.retrieve, "custom")
      ]

  for retriever, method in retrievers:
    print(f"Method {method} for retriever {retirever_name} start!")

    retrievers_top = []

    for answer in tqdm(model_answers['model_answer']):
      answer = answer.replace("<|end_of_text|>", "")
      docs = retriever(answer)
      texts = [doc.page_content for doc in docs]
      retrievers_top.append(texts)

    for i in range(0, 5):
      model_answers["_".join([retirever_name, method, f"top{i}"])] = [context[i] for context in retrievers_top]

    print(f"Retriever {retirever_name} with {method} was added!")



Method faiss for retriever labse start!


100%|██████████| 120/120 [00:02<00:00, 42.95it/s]


Retriever labse with faiss was added!
Method bm25 for retriever labse start!


100%|██████████| 120/120 [00:26<00:00,  4.58it/s]


Retriever labse with bm25 was added!
Method ensemble for retriever labse start!


...

Retriever multi with bm25 was added!
Method ensemble for retriever multi start!


100%|██████████| 120/120 [00:29<00:00,  4.05it/s]


Retriever multi with ensemble was added!
Method custom for retriever multi start!


100%|██████████| 120/120 [00:35<00:00,  3.41it/s]

Retriever multi with custom was added!


In [ ]:
model_answers.head()

,Unnamed: 0,instruction,input,output,model_answer,labse_faiss_top0,labse_faiss_top1,labse_faiss_top2,labse_faiss_top3,labse_faiss_top4,...,multi_ensemble_top0,multi_ensemble_top1,multi_ensemble_top2,multi_ensemble_top3,multi_ensemble_top4,multi_custom_top0,multi_custom_top1,multi_custom_top2,multi_custom_top3,multi_custom_top4
0,2588,Ты - юридический консультант. Тебе даётся след...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ...","Нет, фотофиксация не обязательна, если иные до...",Глава 26. Предмет доказывания. Доказательства....,Глава 6. Доказательства и доказывание Статья 6...,Глава 6. Доказательства и доказывание Статья 6...,"а Российской Федерации, совершенных с использо...",Глава 26. Предмет доказывания. Доказательства....,...,"и фото- и киносъемки, иных установленных спосо...","фото- и киносъемки, иных установленных способо...","анными в статьях 27.2, 27.3 настоящего Кодекса...","владении которого находятся товары, транспортн...",присутствии двух понятых либо с применением ви...,"их доказательств перед другими и определять, к...","ые действия (бездействие), суд может истребова...",е документы не должны содержать ссылки на дока...,"и фото- и киносъемки, иных установленных спосо...",Глава 6. Доказательства и доказывание Статья 7...
1,6365,Ты - юридический консультант. Тебе даётся след...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано...","Да, согласно ч. 1 ст. 30.7 КоАП РФ, суд вправе...",Глава 30. Пересмотр постановлений и решений по...,"ение норм процессуального права, если оно прив...","рации, суда общей юрисдикции о признании недей...",суда первой или апелляционной инстанции и прин...,"суда и заявление о его отмене, подлежит приост...",...,ыть указано только на признание иска и приняти...,"суда и заявление о его отмене, подлежит приост...",дерации определения об отмене этого решения;2)...,о суда в случае вынесения иным судом Российско...,е о выдаче исполнительного листа на принудител...,"творения;2) отменить судебный акт суда первой,...",ыть указано только на признание иска и приняти...,е о выдаче исполнительного листа на принудител...,ях могут быть устранены без возвращения дела н...,и и направить дело на новое рассмотрение в соо...
2,10807,Ты - юридический консультант. Тебе даётся след...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ...","Нет, ст. 5.61 КоАП РФ не предусматривает отягч...",") обстоятельства, установленные при рассмотрен...",Глава 28. Возбуждение дела об административном...,лежащим образом извещенного прокурора не являе...,суда первой или апелляционной инстанции и прин...,"енить судебный акт суда первой, апелляционной ...",...,"рассмотрению судом, в который оно направлено. ...","ящей статьи, устанавливается судом.7. Применен...",ыть указано только на признание иска и приняти...,повлечь за собой наложение судебного штрафа в ...,"уд, давать заведомо ложные показания, отказыва...",Глава 6. Доказательства и доказывание Статья 6...,Глава 6. Доказательства и доказывание Статья 8...,ыть указано только на признание иска и приняти...,"уд, давать заведомо ложные показания, отказыва...",повлечь за собой наложение судебного штрафа в ...
3,1120,Ты - юридический консультант. Тебе даётся след...,"На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ...","По п. 21 ст. 12 Закона об ОСАГО, за каждый ден...",выплаты или срока выдачи потерпевшему направле...,ветствии с пунктом 4 статьи 10 настоящего Феде...,такому лицу путем перечисления суммы компенсац...,нь просрочки в размере одного процента от опре...,твии с законодательством о несостоятельности (...,...,выплаты или срока выдачи потерпевшему направле...,астоящим Федеральным законом суммы страхового ...,нь просрочки в размере одного процента от опре...,такому лицу путем перечисления суммы компенсац...,им пунктом неустойка (пеня) или сумма финансов...,такому лицу путем 

# Подсчёт BertScore

In [ ]:
# %pip install bert_score


In [ ]:
from bert_score import BERTScorer

scorer = BERTScorer(
    model_type="ai-forever/ru-en-RoSBERTa",
    num_layers=20,
    lang="ru"
)

def bertscorer(str1, str2):
  return float(scorer.score([str1], [str2])[2])

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm
import heapq
import numpy as np

retrive_estimates = {
    'labse_faiss': [] ,
    'labse_ensemble': [],
    'labse_retrieve': [],
    'labse_bm25': [],

    'mini_faiss': [],
    'mini_ensemble': [],
    'mini_retrieve': [],
    'mini_bm25': [],

    'multi_faiss': [],
    'multi_ensemble': [],
    'multi_retrieve': [],
    'multi_bm25': [],
}

for n in tqdm(range(len(model_answers))):

  answer = model_answers['model_answer'][n]

  retrive = {
    'labse_faiss': [model_answers[f"labse_faiss_top{i}"][n] for i in range(5)],
    'labse_ensemble': [model_answers[f"labse_ensemble_top{i}"][n] for i in range(5)],
    'labse_retrieve': [model_answers[f"labse_custom_top{i}"][n] for i in range(5)],
    'labse_bm25': [model_answers[f"labse_bm25_top{i}"][n] for i in range(5)],

    'mini_faiss': [model_answers[f"mini_faiss_top{i}"][n] for i in range(5)],
    'mini_ensemble': [model_answers[f"mini_ensemble_top{i}"][n] for i in range(5)],
    'mini_retrieve': [model_answers[f"mini_custom_top{i}"][n] for i in range(5)],
    'mini_bm25': [model_answers[f"mini_bm25_top{i}"][n] for i in range(5)],

    'multi_faiss': [model_answers[f"multi_faiss_top{i}"][n] for i in range(5)],
    'multi_ensemble': [model_answers[f"multi_ensemble_top{i}"][n] for i in range(5)],
    'multi_retrieve': [model_answers[f"multi_custom_top{i}"][n] for i in range(5)],
    'multi_bm25': [model_answers[f"multi_bm25_top{i}"][n] for i in range(5)]
  }

  for method in retrive:

    scores = []
    for text in retrive[method]:

      F1 = bertscorer(answer, text)
      scores.append(F1)

    retrive_estimates[method].append(np.mean(scores).round(3))

retriver_scores_df = pd.DataFrame(retrive_estimates)

100%|██████████| 120/120 [04:32<00:00,  2.27s/it]


In [ ]:
retriver_scores_df.median()

,0
labse_faiss,0.7070
labse_ensemble,0.7070
labse_retrieve,0.7020
labse_bm25,0.7005
mini_faiss,0.6700
mini_ensemble,0.6700
mini_retrieve,0.6905
mini_bm25,0.7005
multi_faiss,0.6960
multi_ensemble,0.6960


In [ ]:
retriver_scores_df.median()

,0
labse_faiss,0.7070
labse_ensemble,0.7070
labse_retrieve,0.7025
labse_bm25,0.7005
mini_faiss,0.6700
mini_ensemble,0.6700
mini_retrieve,0.6890
mini_bm25,0.7005
multi_faiss,0.6960
multi_ensemble,0.6960


In [ ]:
# !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
# !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
# !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
finetune_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./Lora",
    max_seq_length = 8096,
    dtype = None,
    load_in_4bit = True,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
prompt_format = """Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим дополнительный контекст. Напишите ответ, который соответствующим образом завершает запрос.

### Инструкция:
{}

### Ввод:
{}

### Опирайся на ниже приведённый контекст для ответа на заданный вопрос:
{}

### Ответ:
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    retriever_contexts = [" ".join([instruction, input]) for instruction, input in zip(instructions, inputs)]
    contexts = [[
        context.page_content
        for context in custom_retriever_labse.ensemble_retrieve(retriver_input)[:2]
        ]
                for retriver_input in retriever_contexts]
    texts = []
    for instruction, input, context in zip(instructions, inputs, contexts):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_format.format(instruction, input, "\n".join(context))
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
from datasets import Dataset

In [ ]:
test_df = Dataset.from_pandas(model_answers)

In [ ]:
test_df = test_df.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
# Добавление ответа в базу
def add_to_output_data(text):
    response_start = text.find("### Ответ:") + len("### Ответ:")
    #response_end = text.find("<|end_of_text|>")
    answer = text[response_start:].strip()

    return answer

# Красивый вывод одного запроса
def print_one_out(out_data, n):

    cols= out_data.columns

    for i in range(4):
        print(out_data.columns[i], '\n')
        print(out_data.iloc[n, i], '\n\n')

In [ ]:
from tqdm import tqdm
new_column = []
for item in tqdm(test_df):

    # Запрос в модель
    inputs = tokenizer(
    [
      item['text']
    ], return_tensors = "pt").to("cuda")

    outputs = finetune_model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    out = tokenizer.batch_decode(outputs)


    # Запись ответа
    answer = add_to_output_data(out[0])
    new_column.append(answer)

 20%|██        | 24/120 [04:06<16:31, 10.33s/it]Unsloth: Input IDs of length 13033 > the model's max sequence length of 8096.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 53%|█████▎    | 64/120 [13:03<11:09, 11.95s/it]Unsloth: Input IDs of length 13202 > the model's max sequence length of 8096.
We shall truncate it ourselves. It's imperative if you correct this issue first.
100%|██████████| 120/120 [23:37<00:00, 11.81s/it]


In [ ]:
new_column[1]

'Нет, отсутствие мотивированного решения инспектора ГИБДД не является основанием для отмены постановления. Суды исходят из того, что должностное лицо обязано мотивировать свои действия, а если это не сделано, производство прекращается (ч. 1 ст. 29.10 КоАП РФ). Однако если суд установит, что отсутствие мотивации не повлияло на исход дела, отмена постановления маловероятна.<|end_of_text|>'

In [ ]:
model_answers['output'][1]

'Да, могут. Согласно ст. 29.10 КоАП РФ, постановление по делу об административном правонарушении должно содержать мотивированное решение с оценкой доказательств. Если должностное лицо не привело обоснование вывода об отсутствии состава правонарушения (п. 2 ч. 1 ст. 24.5 КоАП РФ), суд вправе отменить такое постановление.'

In [ ]:
model_answers['context_answer']= new_column

In [ ]:
model_answers.head()

,Unnamed: 0,instruction,input,output,model_answer,context_answer
0,2588,Ты - юридический консультант. Тебе даётся след...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ...","Нет, фотофиксация не обязательна, если иные до...","Нет, не обязаны. Фотофиксация применяется толь..."
1,6365,Ты - юридический консультант. Тебе даётся след...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано...","Да, согласно ч. 1 ст. 30.7 КоАП РФ, суд вправе...","Нет, отсутствие мотивированного решения инспек..."
2,10807,Ты - юридический консультант. Тебе даётся след...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ...","Нет, ст. 5.61 КоАП РФ не предусматривает отягч...","Нет, если оскорбление было направлено на потер..."
3,1120,Ты - юридический консультант. Тебе даётся след...,"На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ...","По п. 21 ст. 12 Закона об ОСАГО, за каждый ден...",Неустойка за задержку страхового возмещения ра...
4,204,Ты - юридический консультант. Тебе даётся след...,Может ли страховая компания взыскать выплаченн...,"Да, может. Согласно п. 1 ст. 14 Федерального з...","Да, согласно п. «г» ч. 1 ст. 14 Федерального з...","Да, страховая компания вправе взыскать выплаче..."


In [ ]:
model_answers.to_csv("model_answers_context")

In [ ]:
from tqdm import tqdm
import heapq
import numpy as np

retrive_estimates = {
    'lora': [] ,
    'rag': []
}

for n in tqdm(range(len(model_answers))):

  for method in retrive_estimates:

    if method == 'lora':
      F1 = bertscorer(model_answers['output'][n], model_answers['model_answer'][n])
    else:
      F1 = bertscorer(model_answers['output'][n], model_answers['context_answer'][n])

    retrive_estimates[method].append(F1)

retriver_scores_df = pd.DataFrame(retrive_estimates)

100%|██████████| 120/120 [00:09<00:00, 12.66it/s]


In [ ]:
retriver_scores_df.mean()

,0
lora,0.815801
rag,0.784183
